In [1]:
import librosa
import numpy as np
import os
# Set the duration and hop length for audio processing
duration = 5  # seconds
hop_length = 512

# Define a function to load and extract LFCC features from an audio file
def extract_lfcc_features(file_path):
    # Load the audio file and extract the LFCC features
    y, sr = librosa.load(file_path, duration=duration)
    lfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=hop_length)
    return lfcc

# Define a function to load and preprocess the dataset
def load_dataset(dataset_path):
    # Load the audio files and extract the LFCC features
    lfcc_features = []
    labels = []
    for label, class_name in enumerate(sorted(os.listdir(dataset_path))):
        class_path = os.path.join(dataset_path, class_name)
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            lfcc = extract_lfcc_features(file_path)
            lfcc_features.append(lfcc)
            labels.append(label)
    # Convert the LFCC features and labels to numpy arrays
    lfcc_features = np.array(lfcc_features)
    labels = np.array(labels)
    # Normalize the LFCC features
    mean = np.mean(lfcc_features, axis=0)
    std = np.std(lfcc_features, axis=0)
    np.save('mean.npy', mean)
    np.save('std.npy', std)
    lfcc_features -= np.mean(lfcc_features, axis=0)
    lfcc_features /= np.std(lfcc_features, axis=0)
    
    return lfcc_features, labels


# Load the dataset
dataset_path = '/kaggle/input/detect-cry/data'
lfcc_features, labels = load_dataset(dataset_path)

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Load the data
#lfcc_features = np.load('lfcc_features.npy')
#labels = np.load('labels.npy')

# Normalize the data
lfcc_features /= np.std(lfcc_features, axis=0)
num_classes=3
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(lfcc_features, labels, test_size=0.2, random_state=42)

# Convert the labels to categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train = X_train.reshape(X_train.shape[0], 13, 216, 1)
X_test = X_test.reshape(X_test.shape[0], 13, 216, 1)
# Define the CNN architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(13, 216, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 100
batch_size = 32
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.summary()
model.save('detection.h5');

Epoch 1/100
5/5 [==============================] - 2s 132ms/step - loss: 0.7831 - accuracy: 0.6250 - val_loss: 0.5087 - val_accuracy: 0.6944
Epoch 2/100
5/5 [==============================] - 0s 54ms/step - loss: 0.4332 - accuracy: 0.6944 - val_loss: 0.4294 - val_accuracy: 0.6944
Epoch 3/100
5/5 [==============================] - 0s 58ms/step - loss: 0.3574 - accuracy: 0.7986 - val_loss: 0.3828 - val_accuracy: 0.8611
Epoch 4/100
5/5 [==============================] - 0s 59ms/step - loss: 0.2882 - accuracy: 0.9028 - val_loss: 0.3170 - val_accuracy: 0.7778
Epoch 5/100
5/5 [==============================] - 0s 55ms/step - loss: 0.1909 - accuracy: 0.9514 - val_loss: 0.2260 - val_accuracy: 1.0000
Epoch 6/100
5/5 [==============================] - 0s 66ms/step - loss: 0.1149 - accuracy: 0.9931 - val_loss: 0.1647 - val_accuracy: 0.9444
Epoch 7/100
5/5 [==============================] - 0s 57ms/step - loss: 0.0554 - accuracy: 1.0000 - val_loss: 0.1024 - val_accuracy: 0.9722
Epoch 8/100
5/5 [==

In [3]:
import librosa
import numpy as np
import requests
from keras.models import load_model

# Load the trained model
model = load_model('/kaggle/working/detection.h5')

# Load audio file
y, sr = librosa.load('/kaggle/input/detect-cry/data/Silence/silence.wav_40.wav', sr=16000)

# Extract MFCC features
mfcc_features = librosa.feature.mfcc(y=y, sr=sr, n_fft=1024, hop_length=372,n_mfcc=13)
mfcc_features = mfcc_features.T

# Reshape data for use with the model
#num_frames = 13  # change this to match the number of frames in your data
#num_features = 87
print(mfcc_features.shape)
mfcc_features = mfcc_features.reshape(1, 13, 216, 1)

# Make a prediction using the trained model
prediction = model.predict(mfcc_features)
#prediction_class = np.argmax(prediction)

# Print the predicted class
#classes = ['Hunger', 'Discomfort', 'Sleepiness', 'Pain', 'None']
#print('Predicted class:', classes[prediction_class])
print(prediction)
class_labels = ['lAUGH','Silence','Crying']
predicted_label = class_labels[np.argmax(prediction)]

# Print the predicted class label
print(predicted_label)


(216, 13)
1/1 [==============================] - 0s 140ms/step
[[0. 0. 1.]]
Crying
